# AIMS Watershed Query: AnnAGNPS files preparation
This notebook replicates the data flow of what a user would do when clicking on the AIMS web interface

##### Main library imports

In [1]:
from pathlib import Path
import json
import itertools

import pyagnps

import geopandas as gpd
import pandas as pd

from tqdm import tqdm

from sqlalchemy import URL, create_engine, text as sql_text
from sqlalchemy.orm import sessionmaker

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

## User input here

### Outlet coordinates

In [2]:
# lon, lat = -89.92829, 32.52256 # Upper Pearl River upstream of reservoir
lon, lat = -89.96793, 32.38881 # Pelahatchie Creek

### Time bounds and climate method

In [3]:
climate_method = 'nldas2' # valid options ares 'nldas2', 'cmip5'

# Time period in YYYY-MM-DD format (both bounds included)
startDate="2010-01-01" 
endDate="2020-12-31"

### Main output directory

In [4]:
output_folder = Path('C:/Users/Luc/Desktop/pelahatchie_creek')

### Paths to useful static files

In [133]:
# Database credentials
credentials = Path("../../inputs/db_credentials.json")

# GeoPackage that contains the centroid of each NLDAS-2 grid
path_nldas_grid_centroids = Path("D:/AIMS/Datasets/Climate/NLDAS2/NLDAS2_GRID_CENTROIDS_epsg4326.gpkg")
# path_nldas_grid_centroids = Path("C:/Users/Luc/projects/pyagnps/inputs/climate/NLDAS2_GRID_CENTROIDS_epsg4326.gpkg")

# Path to root folder containing variables of the CMIP5 MACAv2METDATA
path_to_cmip5_historical_and_rcp45 = Path("D:/AIMS/Datasets/Climate/CMIP/CMIP5/MACAv2METDATA/CNRM-CM5/r1i1p1/")
path_to_cmip5_raster_points_clim_id = path_to_cmip5_historical_and_rcp45 / "cmip5_maca_v2_metdata_pts_clim_ids.gpkg"

# Path to SCS Storm Types
path_to_scs_storm_types = Path('D:/AIMS/Datasets/Climate/TR-55/scs_storm_types.gpkg')

# Path to RUSLE2 Precipitation Zones
path_to_precip_zones = Path('D:/AIMS/Datasets/Management/RUSLE2_Climate/RUSLE2-Climate-Data-20231117/precip_zones_RUSLE2_cleaned_manually_extrapolated_pchip_linear_US_units.gpkg')


## Main process starts here

### Setting up database connection

In [6]:
# DATABASE SETUP
with open(credentials, "r") as f:
    credentials = json.load(f)

user = credentials["user"]
password = credentials["password"]
host = credentials["host"]
port = credentials["port"]
database = credentials["database"]

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

### Define output directories

In [7]:
output_folder.mkdir(exist_ok=True)

def make_annagnps_inputs_dirs(output_folder, subdirs=['general', 'climate', 'simulation', 'watershed', 'GIS']):
    subdirs_paths = []
    for subdir in subdirs:
        category_dir = output_folder / subdir
        category_dir.mkdir(exist_ok=True)
        subdirs_paths.append(category_dir)
    return subdirs_paths
        

general_dir, \
climate_dir, \
simulation_dir, \
watershed_dir, \
gis_dir = make_annagnps_inputs_dirs(output_folder)

### Identify T-HUC containing the watershed of interest

In [8]:
thuc = pd.read_sql_query(sql_text(f"SELECT thuc_near_run_id_tr({lon},{lat})"),con=engine.connect())
thuc_id = thuc.iloc[0].values[0]

### Query reach and cell data sections along with their geometries

#### Cells geometry

In [9]:
cells_query = f"SELECT geom, cell_id FROM thuc_cell_geo_tr({lon},{lat}, '{thuc_id}')"

cells_geometry = gpd.read_postgis(sql=sql_text(cells_query), con=engine.connect(), geom_col='geom')
cells_list = cells_geometry['cell_id'].unique() # List of cell_ids


#### Reach Geometry

In [10]:
reaches_query = f"SELECT geom, reach_id FROM thuc_reach_geo_tr({lon},{lat}, '{thuc_id}')"

reaches_geometry = gpd.read_postgis(sql=sql_text(reaches_query), con=engine.connect(), geom_col='geom')
reaches_list = reaches_geometry['reach_id'].unique()

#### Cell Data Section

In [11]:
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_data_section WHERE cell_id in {*cells_list,}"

df_cells = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

#### Reach Data Section

In [12]:
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_reach_data_section WHERE reach_id in {*reaches_list,}"

df_reaches = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

Make reach data section "valid" for AnnAGNPS i.e. add an "OUTLET" row at the top

In [13]:
def make_df_reaches_valid(df_reaches):
    reaches = set(df_reaches['reach_id'])
    receiving_reaches = set(df_reaches['receiving_reach'])

    outlet_reach = list(receiving_reaches - reaches)[0]
    # print(f"Outlet reach: {outlet_reach}")

    outlet_row = df_reaches[df_reaches['receiving_reach']==outlet_reach].copy()
    outlet_row['reach_id'] = outlet_reach
    outlet_row['receiving_reach'] = 'OUTLET'
    outlet_row['length'] = 0

    df_reaches_valid = pd.concat([outlet_row, df_reaches], ignore_index=True)
    return df_reaches_valid

In [14]:
df_reaches_valid = make_df_reaches_valid(df_reaches)

#### Join data sections with their respective geometries

In [15]:
cells_geometry = cells_geometry.merge(df_cells, on='cell_id')
reaches_geometry = reaches_geometry.merge(df_reaches, on='reach_id')

### Query Soil data

Query soil_data and soil_layers_daya for matching soil_id as well as raw soil data

In [16]:
soil_ids_list = df_cells['soil_id'].unique()

In [17]:
query_soil = f"""SELECT * FROM usa_valid_soil_data WHERE "Soil_ID" in {*soil_ids_list,}"""
query_soil_layers = f"""SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" in {*soil_ids_list,}"""
query_raw = f"""SELECT * FROM raw_nrcs_soil_data WHERE "mukey" in {*soil_ids_list,}"""

df_soil_data = pd.read_sql_query(sql=sql_text(query_soil), con=engine.connect())
df_soil_layers_data = pd.read_sql_query(sql=sql_text(query_soil_layers), con=engine.connect())
df_raw = pd.read_sql_query(sql=sql_text(query_raw), con=engine.connect())

### Query Management Data

#### Get Management Field IDs

In [18]:
mgmt_field_ids_list = df_cells['mgmt_field_id'].unique()

In [19]:
query_mgmt_field_data = f"""SELECT * FROM annagnps_mgmt_field WHERE "Field_ID" in {*mgmt_field_ids_list,}"""
df_mgmt_field = pd.read_sql_query(sql=sql_text(query_mgmt_field_data), con=engine.connect())
df_mgmt_field.head()

,Field_ID,Landuse_Type_ID,Mgmt_Schd_ID,Greg_Yr_for_1st_Yr_of_Rotation,Percent_Rock_Cover,Interrill_Erosion_Code,Random_Roughness,Terrace_Horizontal_Distance,Terrace_Grade,Tile_Drain_ID,Input_Units_Code,CDL_Category,CDL_Value,Modified_CDL_Category
0,Corn,CROPLAND,Corn,None,None,None,None,None,None,None,1,Corn,1,Corn
1,Cotton,CROPLAND,Cotton,None,None,None,None,None,None,None,1,Cotton,2,Cotton
2,Developed_Open_Space,URBAN,Developed_Open_Space,None,None,None,None,None,None,None,1,Developed/Open Space,121,Developed_Open_Space
3,Fallow_Idle_Cropland,PASTURE,Fallow_Idle_Cropland,None,None,None,None,None,None,None,1,Fallow/Idle Cropland,61,Fallow_Idle_Cropland
4,Grassland_Pasture,CROPLAND,Grassland_Pasture,None,None,None,None,None,None,None,1,Grassland/Pasture,176,Grassland_Pasture


#### Get matching Management Schedule IDs

In [20]:
mgmt_schedule_ids_list = df_mgmt_field['Mgmt_Schd_ID'].unique()

In [21]:
query_mgmt_field_data = f"""SELECT * FROM annagnps_mgmt_schd WHERE "Mgmt_Schd_ID" in {*mgmt_schedule_ids_list,}"""
df_mgmt_schd = pd.read_sql_query(sql=sql_text(query_mgmt_field_data), con=engine.connect())

#### Get matching Crop, Non-Crop, Management Operation, and Runoff Curve Data

In [22]:
mgmt_crop_ids_list     = df_mgmt_schd['New_Crop_ID'].dropna().unique()
mgmt_non_crop_ids_list = df_mgmt_schd['New_Non-Crop_ID'].dropna().unique()
mgmt_oper_ids_list     = df_mgmt_schd['Mgmt_Operation_ID'].dropna().unique()
roc_ids_list           = df_mgmt_schd['Curve_Number_ID'].dropna().unique()

In [23]:
query_mgmt_crop_data        = f"""SELECT * FROM annagnps_crop WHERE "Crop_ID" in {*mgmt_crop_ids_list,}"""
query_mgmt_crop_growth_data = f"""SELECT * FROM annagnps_crop_growth WHERE "Crop_Growth_ID" in {*mgmt_crop_ids_list,}"""

query_mgmt_non_crop_data    = f"""SELECT * FROM annagnps_non_crop WHERE "Non-Crop_ID" in {*mgmt_non_crop_ids_list,}"""
query_mgmt_oper_data        = f"""SELECT * FROM annagnps_mgmt_oper WHERE "Mgmt_Operation_ID" in {*mgmt_oper_ids_list,}"""

query_roc_data              = f"""SELECT * FROM annagnps_runoff_curve WHERE "Curve_Number_ID" in {*roc_ids_list,}"""

df_mgmt_crop        = pd.read_sql_query(sql=sql_text(query_mgmt_crop_data), con=engine.connect())
df_mgmt_crop_growth = pd.read_sql_query(sql=sql_text(query_mgmt_crop_growth_data), con=engine.connect())
df_mgmt_non_crop    = pd.read_sql_query(sql=sql_text(query_mgmt_non_crop_data), con=engine.connect())
df_mgmt_oper        = pd.read_sql_query(sql=sql_text(query_mgmt_oper_data), con=engine.connect())
df_roc              = pd.read_sql_query(sql=sql_text(query_roc_data), con=engine.connect())

### Watershed scale variables - Simulation Period Data

#### Water boundaries

In [134]:
bounds = cells_geometry[['geom']].copy(deep=True)
bounds.geometry = bounds.geometry.buffer(0.00001)
bounds = bounds.dissolve()
bounds = bounds.to_crs('epsg:4326')

#### Watershed SCS Storm Type

In [117]:
scs_storm_types = gpd.read_file(path_to_scs_storm_types)
scs_storm_types = scs_storm_types.to_crs('epsg:4326')

In [120]:
bounds_scs = bounds.overlay(scs_storm_types)
bounds_scs['area'] = bounds_scs.geometry.area

In [125]:
main_storm_type = bounds_scs.loc[bounds_scs['area'].argmax(), 'SCS Zone Type']

#### 10_Year_EI, EI_Zone, R_factor

In [135]:
precip_zones = gpd.read_file(path_to_precip_zones)

In [137]:
bounds_precip = bounds.overlay(precip_zones)
bounds_precip['area'] = bounds_precip.geometry.area

,RUSLE_REQ,REC_LINK,Shape_Leng,Shape_Area,State,County,Precip_Zone,EI_Zone,Annual_PPT,R_factor,Max_Eros_Density,10_year_24_hr_ppt,10_year_EI,Notes,GEOID,min_ppt_precip_zone,max_ppt_precip_zone,mid_ppt_precip_zone,geometry,area
0,NaN,28123,1.693727,0.151541,Mississippi,Scott,NaN,US_106,57.688610,478.823513,11.817286,6.6,156.024,NaN,28123,NaN,NaN,NaN,"POLYGON ((-89.74364 32.41763, -89.74364 32.417...",0.008451
1,NaN,28121,2.459352,0.199821,Mississippi,Rankin,NaN,US_106,56.440866,470.266014,11.847279,6.7,158.790,NaN,28121,NaN,NaN,NaN,"POLYGON ((-89.95006 32.31314, -89.95006 32.313...",0.043841


In [155]:
weighted_R_fctr = (bounds_precip['area'] * bounds_precip['R_factor']).sum() / bounds_precip['area'].sum()
weighted_10_year_EI = (bounds_precip['area'] * bounds_precip['10_year_EI']).sum() / bounds_precip['area'].sum()

dominant_EI = bounds_precip.loc[bounds_precip['area'].argmax(), 'EI_Zone']

if dominant_EI == 'default':
    dominant_EI = pyagnps.constants.DEFAULT_EI_NUMBER # 100 
else:
    dominant_EI = int(dominant_EI.replace('US_',''))

106

### Generate climate files

Get watershed centroid computed in terms of lat and lon

In [24]:
# /!\ This centroid is computed in degrees coordinates
lon0, lat0 = cells_geometry.dissolve().centroid.x[0], cells_geometry.dissolve().centroid.y[0] 

Reset cells secondary climate file id (just in case)

In [25]:
cells_geometry['secondary_climate_file_id'] = None

#### Using NLDAS-2

Identify the NLDAS-2 grid ID nearest to each cell and populate it in the `secondary_climate_file_id`

In [26]:
if climate_method == 'nldas2':
    nldas_centroids = gpd.read_file(path_nldas_grid_centroids)
    cells_geometry = cells_geometry.sjoin_nearest(nldas_centroids)
    cells_geometry['secondary_climate_file_id'] = cells_geometry['nldas2_grid_ID']
    cells_geometry.drop(columns=['nldas2_grid_ID', 'index_right'], inplace=True)

    secondary_climate_ids = cells_geometry.loc[:,['cell_id', 'secondary_climate_file_id']]\
                            .drop_duplicates(subset='cell_id')\
                            .set_index('cell_id')
    
    # Generate climate data for the unique NLDAS-2 grid ID featuring in the watershed 
    climate_station_points = nldas_centroids[nldas_centroids['nldas2_grid_ID'].isin(cells_geometry['secondary_climate_file_id'].unique())]
    climate_station_points = climate_station_points.to_crs('epsg:4326')

    # Update df_cells with data in secondary_climate_ids
    if df_cells.index.name != 'cell_id':
        df_cells = df_cells.set_index('cell_id')

    df_cells.update(secondary_climate_ids)
    df_cells = df_cells.reset_index()

#### Using CMIP5 data

Load CMIP5 data

In [27]:
if climate_method == 'cmip5':
    clm = pyagnps.climate.ClimateAnnAGNPSCoords(coords=None,
                                                start=startDate,
                                                end=endDate)

    clm.read_cmip5_maca_data(path_to_cmip5_historical_and_rcp45.glob("*/*.nc"))

Generate or load geopackage file containing the centroids of each raster points of the CMIP5 dataset

In [28]:
if climate_method == 'cmip5':
    if path_to_cmip5_raster_points_clim_id.exists():
        print('Reading points and clim_id of CMIP5 MACAv2METDATA')
        cmip_pts = gpd.read_file(path_to_cmip5_raster_points_clim_id)
    else:
        # Generate all the possible pairs of points
        print('Generating points and clim_id of CMIP5 MACAv2METDATA')
        lat_vals = clm.ds['lat'].values
        lon_vals = clm.ds['lon'].values - 360 # For CMIP6 and CMIP5 the longitude needs to be in the [0, 360[ range, here we bring it back in the -180, 180

        all_lon_lat_pairs = list(itertools.product(lon_vals, lat_vals))

        lon_values, lat_values = zip(*all_lon_lat_pairs)

        pts = gpd.points_from_xy(x=lon_values, y=lat_values, crs=4326)
        cmip_pts = gpd.GeoDataFrame(geometry=pts, crs=4326)

        cmip_pts['clim_id'] = cmip_pts['geometry'].apply(lambda geom: clm.generate_cmip_lon_lat_secondary_climate_id((geom.x, geom.y)))

        cmip_pts.to_file(path_to_cmip5_raster_points_clim_id, driver='GPKG')

Assigning secondary climate id for every cell based on the CMIP5 data

In [29]:
if climate_method == 'cmip5':

    cells_geometry = cells_geometry.sjoin_nearest(cmip_pts)
    cells_geometry['secondary_climate_file_id'] = cells_geometry['clim_id']
    cells_geometry.drop(columns=['clim_id', 'index_right'], inplace=True)


    secondary_climate_ids = cells_geometry.loc[:,['cell_id', 'secondary_climate_file_id']]\
                                .drop_duplicates(subset='cell_id')\
                                .set_index('cell_id')
    
    # List of points for which to extract the climate data for this watershed
    climate_station_points = cmip_pts[cmip_pts['clim_id'].isin(secondary_climate_ids['secondary_climate_file_id'])]
    climate_station_points = climate_station_points.to_crs('epsg:4326')

    # Update df_cells with data in secondary_climate_ids
    if df_cells.index.name != 'cell_id':
        df_cells = df_cells.set_index('cell_id')

    df_cells.update(secondary_climate_ids)
    df_cells = df_cells.reset_index()

#### Generate climate data dictionary for selected climate source

In [30]:
climate_data = {}

for feature in tqdm(climate_station_points.iterfeatures(), total=len(climate_station_points)):

    x, y = feature['geometry']['coordinates']

    if climate_method == 'nldas2':
        clim_id = feature['properties']['nldas2_grid_ID']
        climate_station_name = f"NLDAS-2 Grid ID {clim_id}"

        clm = pyagnps.climate.ClimateAnnAGNPSCoords(coords=(x,y), start=startDate, end=endDate, date_mode="local")
        df = clm.query_nldas2_generate_annagnps_climate_daily()
        
    elif climate_method == 'cmip5':
        clim_id = feature['properties']['clim_id']
        climate_station_name = f"CMIP5 MACAv2METDATA raster ID {clim_id}"
        
        clm.update_coords_start_end_dates(coords=(x,y), start=startDate, end=endDate, date_mode="local")
        df = clm.generate_annagnps_daily_climate_data_cmip5_maca()
        

    climate_data[clim_id] = {
        'data': df, # DataFrame containing the climate data
        'climate_station': { # Climate station metadata
            'output_filepath': climate_dir / f'climate_station_{clim_id}.csv',
            'climate_station_name': climate_station_name,
            'beginning_climate_date': clm.start.strftime("%m/%d/%Y"),
            'ending_climate_date': clm.end.strftime("%m/%d/%Y"),
            'latitude': y,
            'longitude': x,
            'elevation': cells_geometry.loc[cells_geometry['secondary_climate_file_id'] == clim_id, 'avg_elevation'].mean()
            }
    }

# A "global" climate daily / station data needs to be produced so the last climate file will be used
climate_data['global'] = climate_data[clim_id]
climate_data['global']['climate_station']['output_filepath'] = climate_dir / 'climate_station.csv'

100%|██████████| 8/8 [00:14<00:00,  1.80s/it]


## Export everything to files

#### Reaches and Cells

Data sections

In [31]:
df_cells.to_csv(watershed_dir / 'cell_data_section.csv', index=False)
df_reaches_valid.to_csv(watershed_dir / 'reach_data_section.csv', index=False)

GIS layers

In [32]:
cells_geometry.to_file(gis_dir / 'cells_geometry.gpkg', driver='GPKG', index=False)
reaches_geometry.to_file(gis_dir / 'reaches_geometry.gpkg', driver='GPKG', index=False)

#### Soil Data Section

In [33]:
df_soil_data.to_csv(general_dir / 'soil_data.csv', index=False)
df_soil_layers_data.to_csv(general_dir / 'soil_layers_data.csv', index=False)
df_raw.to_csv(general_dir / 'raw_soil_data_gNATSGO.csv', index=False)

#### Management Data

In [34]:
df_mgmt_crop.to_csv(general_dir / 'crop_data.csv', index=False)
df_mgmt_crop_growth.to_csv(general_dir / 'crop_growth.csv', index=False)
df_mgmt_non_crop.to_csv(general_dir / 'non_crop.csv', index=False)

df_mgmt_field.to_csv(general_dir / 'management_field.csv', index=False)
df_mgmt_schd.to_csv(general_dir / 'management_schedule.csv', index=False)
df_mgmt_oper.to_csv(general_dir / 'management_oper.csv', index=False)


df_roc.to_csv(general_dir / 'runoffcurve.csv', index=False)

#### Climate

In [35]:
for clim_id in climate_data:
    # Climate Daily
    if clim_id == 'global':
        climate_path = climate_dir / f"climate_daily.csv"
    else:
        climate_path = climate_dir / f"climate_daily_{clim_id}.csv"

    climate_data[clim_id]['data'].to_csv(climate_path, index=False, float_format="%.3f")
    # Climate Station
    pyagnps.climate.generate_climate_station_file(**climate_data[clim_id]['climate_station'])

#### Global IDs Factors and Flag Data

In [130]:
DEFAULT_GLOBAL_FACTORS_FLAGS = pyagnps.constants.DEFAULT_GLOBAL_FACTORS_FLAGS
DEFAULT_GLOBAL_FACTORS_FLAGS['Wshd_Storm_Type_ID'] = main_storm_type

In [131]:
globfac_path = simulation_dir / 'globfac.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_GLOBAL_FACTORS_FLAGS, globfac_path)

#### Output Options - Global

In [40]:
DEFAULT_OUTPUT_OPTIONS_GLOBAL = pyagnps.constants.DEFAULT_OUTPUT_OPTIONS_GLOBAL

outopts_global_path = simulation_dir / 'outopts_global.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_OUTPUT_OPTIONS_GLOBAL, outopts_global_path)

#### Output Options - AA

In [37]:
DEFAULT_OUTPUT_OPTIONS_AA = pyagnps.constants.DEFAULT_OUTPUT_OPTIONS_AA

outopts_aa_path = simulation_dir / 'outopts_aa.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_OUTPUT_OPTIONS_AA, outopts_aa_path)

#### Output Options - TABLE

In [38]:
DEFAULT_OUTPUT_OPTIONS_TBL = pyagnps.constants.DEFAULT_OUTPUT_OPTIONS_TBL

outopts_tbl_path = simulation_dir / 'outopts_tbl.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_OUTPUT_OPTIONS_TBL, outopts_tbl_path)

#### AnnAGNPS ID file

In [39]:
DEFAULT_ANNAGNPS_ID = pyagnps.constants.DEFAULT_ANNAGNPS_ID

annaid_path = simulation_dir / 'annaid.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_ANNAGNPS_ID, annaid_path)

#### Simulation Period Data

In [177]:
DEFAULT_SIM_PERIOD_DATA = pyagnps.constants.DEFAULT_SIM_PERIOD_DATA

DEFAULT_SIM_PERIOD_DATA['Simulation_Begin_Year'] = clm.start.year
DEFAULT_SIM_PERIOD_DATA['Simulation_Begin_Month'] = clm.start.month
DEFAULT_SIM_PERIOD_DATA['Simulation_Begin_Day'] = clm.start.day

DEFAULT_SIM_PERIOD_DATA['Simulation_End_Year'] = clm.end.year
DEFAULT_SIM_PERIOD_DATA['Simulation_End_Month'] = clm.end.month
DEFAULT_SIM_PERIOD_DATA['Simulation_End_Day'] = clm.end.day

DEFAULT_SIM_PERIOD_DATA['Rainfall_Fctr'] = weighted_R_fctr
DEFAULT_SIM_PERIOD_DATA['10-Year_EI'] = weighted_10_year_EI
DEFAULT_SIM_PERIOD_DATA['EI_Number'] = dominant_EI

sim_period_path = simulation_dir / 'sim_period.csv'

pyagnps.utils.write_csv_from_dict(DEFAULT_SIM_PERIOD_DATA, sim_period_path)

#### AnnAGNPS Master File